In [11]:
import tensorflow as tf
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn import metrics

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM,Dense,Flatten,Dropout

df = pd.read_json("../input/milling-dataset/ipt_12.json")
df.head()

In [12]:
print(df.shape)
print(df.columns)

for col in df.columns:
    if df[col].dtypes == 'object':
        count = 0
        for val in df[col]:
            nan_sum = 0
            nan_sum = np.isnan(val).sum()
            count += nan_sum
        print(col+": ", df[col].dtypes, ",", count)
    else:
        print(col+": ", df[col].dtypes, ",", df[col].isna().sum(axis=0))
        
for col in df.columns:
    if df[col].dtypes != 'object':
        print(col+": ", df[col].unique())
        
df.dropna(axis=0, how="any", inplace=True)
print(df.shape)

target=df["VB"]
df = df[df.columns.drop("VB")]
print(df.shape)

df.reset_index()

In [13]:
df1=df.drop(["smcDC","vib_table","vib_spindle","AE_table","AE_spindle"],axis=1)
df1=df1.explode("smcAC")    
df2=df.drop(['case', 'run',   'DOC', 'time', 'material', 'feed', 'smcAC',
        'vib_table', 'vib_spindle', 'AE_table', 'AE_spindle'],axis=1)
df2=df2.explode("smcDC")
df3=df.drop(['case', 'run',   'DOC', 'time', 'material', 'feed', 'smcAC',
       'smcDC', 'vib_spindle', 'AE_table', 'AE_spindle'],axis=1)
df3=df3.explode("vib_table")
df4=df.drop(['case', 'run',   'DOC', 'time', 'material', 'feed', 'smcAC',
       'smcDC', 'vib_table',  'AE_table', 'AE_spindle'],axis=1)
df4=df4.explode("vib_spindle")
df5=df.drop(['case', 'run',   'DOC', 'time', 'material', 'feed', 'smcAC',
       'smcDC', 'vib_table', 'vib_spindle', 'AE_spindle'],axis=1)
df5=df5.explode("AE_table")
df6=df.drop(['case', 'run',   'DOC', 'time', 'material', 'feed', 'smcAC',
       'smcDC', 'vib_table', 'vib_spindle', 'AE_table',],axis=1)
df6=df6.explode("AE_spindle")

df1['smcDC']=df2['smcDC']
df1['vib_table']=df3['vib_table']
df1['vib_spindle']=df4['vib_spindle']
df1['AE_table']=df5['AE_table']
df1['AE_spindle']=df6['AE_spindle']

In [14]:
df1.head()

In [15]:
l=np.zeros((146,9000,12))
for i in range(146):
    for j in range(9000):
        ser = df1.iloc[j]
        l1 = ser.to_numpy(dtype=np.float32)
        l[i][j] = l1

In [16]:
l.shape

In [17]:
y = target.to_numpy()
print(y)

In [18]:
X_train, X_val, y_train, y_val = train_test_split(l, y, test_size=0.2, random_state=42)
X_train, X_test, y_train, y_test = train_test_split(X_train, y_train, test_size=0.2)

In [19]:
model = Sequential()
model.add(LSTM(100,
            return_sequences=True,
            input_shape=(X_train.shape[1],12)))
model.add(LSTM(200, return_sequences=True))
model.add(Flatten())
model.add(Dropout(0.5))
model.add(Dense(100, activation='relu'))
#model.add(LSTM(20))
model.add(Dense(1))

model.compile(optimizer='adam', loss="mean_squared_error")

In [20]:
history = model.fit(X_train,y_train,epochs=100, validation_data=(X_val, y_val), validation_steps = 20 ,batch_size=9000)

y_pred = model.predict(X_test)
# y_pred = np.where(y_pred > 0.5,1,0)

In [21]:
print(y_pred)

In [22]:
y_pred = y_pred.flatten()
# cm = metrics.confusion_matrix(y_test, y_pred[:,0])
mse = metrics.mean_squared_error(y_test, y_pred)
# print(cm)
print(mse)